In [ ]:
! unzip "drive/My Drive/4th-cupoy-ml-100-marathon-finalexam.zip"

Archive:  drive/My Drive/4th-cupoy-ml-100-marathon-finalexam.zip
replace image_data/test/0028624c49b3e0610ff9f1d111f5d532.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [23]:
!nvidia-smi
import os
import keras
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Sat Jul 25 05:54:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    71W / 149W |   8681MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
import pandas as pd
import numpy as np
import glob

path = ["image_data/train/daisy/*.jpg",
        "image_data/train/dandelion/*.jpg",
        "image_data/train/rose/*.jpg",
        "image_data/train/sunflower/*.jpg",
        "image_data/train/tulip/*.jpg"
        ]
        
def list_data(path, target=None, mode="train"):
    
    
    if mode == "train":
        if not isinstance(target, int):
            raise TypeError("target must be int.")

        data = glob.glob(path)
        dataDict = {
            "path": data,
            "target": [target for times in range(len(data))]
        }
        dataDf = pd.DataFrame(dataDict)  
    elif mode == "test":
        data = glob.glob(path)
        dataDict = {
            "path": data
        }
        dataDf = pd.DataFrame(dataDict)
    else:
        return ValueError("mode must be train or test.")
    
    tmp = path.split("/")[-2]
    print("%s 的shape :\n" %tmp, dataDf.shape)
    return dataDf

def load_data(dataPath, targetSize=None):

    if not isinstance(dataPath, str):
        raise TypeError("dataPath must be str.")

    if targetSize == None:
        targetSize = (224, 224)

    if not isinstance(targetSize, tuple):
        raise TypeError("targetSize must be tuple.")

    originImg = keras.preprocessing.image.load_img(dataPath, target_size=targetSize)
    img = keras.applications.resnet_v2.preprocess_input(np.array(originImg))
    print("After preprocessing, Img Shape: ", img.shape)
    return img

daisyDf = list_data(path[0], 0)
dandelionDf = list_data(path[1], 1)
roseDf = list_data(path[2], 2)
sunflowerDf = list_data(path[3], 3)
tulipDf = list_data(path[4], 4)
testDf = list_data("image_data/test/*.jpg", mode="test")

trainAssembleArray = [daisyDf, dandelionDf, roseDf, sunflowerDf, tulipDf]

daisy 的shape :
 (500, 2)
dandelion 的shape :
 (687, 2)
rose 的shape :
 (515, 2)
sunflower 的shape :
 (488, 2)
tulip 的shape :
 (633, 2)
test 的shape :
 (2000, 1)


In [25]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

trainList = []
trainTarget = []
for assemble in trainAssembleArray:
    for path, target in zip(assemble["path"],assemble["target"]):
        img = load_data(path)
        oneHot = to_categorical(target, num_classes=5)
        trainList.append(img)
        trainTarget.append(oneHot)

testList = []
for path in testDf["path"]:
    img = load_data(path)
    testList.append(img)
trainNp = np.array(trainList)
trainTargetNp = np.array(trainTarget)
testNp = np.array(testList)
print("forcing train type to np and look at the shape is : ", trainNp.shape)
print("forcing trainTarget type to np and look at the shape is : ", trainTargetNp.shape)
print("forcing test type to np and look at the shape is : ", testNp.shape)

x_train, x_test, y_train, y_test = train_test_split(trainNp, trainTargetNp, shuffle=True, test_size=0.2)

After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preprocessing, Img Shape:  (224, 224, 3)
After preproc

In [43]:

def buildModel(
    mode="resnet", 
    top=False,
    unitArray=[1024, 256],
    denseActi="relu",
    dropRate=0.5,
    target=5,
    freeze=True
    ):

    if mode != "resnet":
        return AttributeError("mode only be resnet.")

    if top == True:
        resnet50 = keras.applications.resnet_v2.ResNet50V2(include_top=top)
        if freeze == True:
            for layer in resnet50.layers:
                layer.trainable = False
    else:
        resnet50 = keras.applications.resnet_v2.ResNet50V2(include_top=top, input_shape=(224, 224, 3))
        if freeze == True:
            for layer in resnet50.layers:
                layer.trainable = False

        x = keras.layers.Flatten()(resnet50.output)
        for idx, unit in enumerate(unitArray):
            if idx == 0:
                x = keras.layers.Dense(units=unit, activation=denseActi)(x)
                x = keras.layers.Dropout(dropRate)(x)
                x = keras.layers.BatchNormalization()(x)
            else:
                x = keras.layers.Dense(units=unit, activation=denseActi)(x)
                x = keras.layers.Dropout(dropRate)(x)
                x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units=target, activation="softmax")(x)

    model = keras.models.Model(inputs=[resnet50.input], outputs=[x])

    model.summary()
    return model

model = buildModel()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [44]:

paramsDict = {
    "EPOCH":20,
    "BATCH_SIZE":64,
    "CALLBACK_MIN_DELTA":1e-2,
    "CALLBACK_PATIENCE":20,
    "opt":"Adam",
    "loss": "categorical_crossentropy",
    "metric": ["accuracy"]
}



model.compile(
    optimizer=paramsDict["opt"],
    loss=paramsDict["loss"],
    metrics=paramsDict["metric"]
)


bMark = False
for epoch in range(paramsDict["EPOCH"]):

    if bMark == True:
        print("===== early stop =====")
        break

    print("The %d train, Ready Go..." % (epoch+1))
    startIdx = 0
    patience = 0

    for batch_size in range(paramsDict["BATCH_SIZE"], x_train.shape[0], paramsDict["BATCH_SIZE"]):
        history = model.train_on_batch(
            x_train[startIdx:batch_size],
            y_train[startIdx:batch_size],
        )
        print(history)

        if history[0] < paramsDict["CALLBACK_MIN_DELTA"]:
            if patience > paramsDict["CALLBACK_PATIENCE"]:
                bMark = True
            patience += 1     
        
        startIdx += 64

model.evaluate(
    x=x_test,
    y=y_test, 
)


The 1 train, Ready Go...
[2.0800285, 0.21875]
[1.4776107, 0.453125]
[1.102915, 0.65625]
[1.2422278, 0.625]
[1.2225525, 0.625]
[1.2289373, 0.6875]
[1.096792, 0.65625]
[0.59089786, 0.796875]
[0.814327, 0.734375]
[0.78101087, 0.71875]
[0.9075724, 0.71875]
[0.7292925, 0.734375]
[0.7936073, 0.78125]
[0.93001324, 0.71875]
[0.6314776, 0.75]
[0.7672801, 0.734375]
[0.61439806, 0.765625]
[0.94900143, 0.6875]
[0.505288, 0.828125]
[0.60249436, 0.71875]
[0.5657019, 0.8125]
[0.571392, 0.796875]
[0.57423353, 0.859375]
[0.51632047, 0.78125]
[0.9065063, 0.703125]
[0.51429504, 0.828125]
[0.78789604, 0.765625]
[0.5522789, 0.796875]
[0.7035155, 0.71875]
[0.7330983, 0.734375]
[0.51147985, 0.8125]
[0.81124234, 0.71875]
[0.5864037, 0.796875]
[0.46864295, 0.859375]
[0.4697464, 0.828125]
The 2 train, Ready Go...
[0.3660464, 0.875]
[0.36304557, 0.859375]
[0.2341515, 0.9375]
[0.5107409, 0.84375]
[0.32471994, 0.890625]
[0.36995786, 0.859375]
[0.2841948, 0.921875]
[0.29609215, 0.9375]
[0.30084193, 0.890625]
[0.204

[1.0335618860953677, 0.7433628439903259]

In [46]:
result = model.predict(testNp)

In [73]:
_target = pd.Series(result.argmax(-1))
_target

0       2
1       0
2       1
3       1
4       2
       ..
1995    0
1996    4
1997    0
1998    4
1999    4
Length: 2000, dtype: int64

In [78]:
import copy
_testDf = copy.copy(testDf)
_testDf["path"] = _testDf["path"].apply(lambda x: x.split("/")[-1].split(".")[0])

finalDf = pd.concat([_testDf, _target], axis=1)
finalDf = finalDf.rename(columns={"path":"Id", 0:"flower_class"})
finalDf.to_csv("./final.csv", encoding="utf-8", index=False)